In [2]:
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
import requests

In [3]:
import requests
import pandas as pd

def fetch_yield_curve_data(url):
    # Fetch the webpage content
    response = requests.get(url)
    if response.status_code == 200:
        # Use pandas to read tables from the webpage content
        dfs = pd.read_html(response.content)
        
        # Check if there are at least two tables
        if len(dfs) >= 2:
            # Adjusted to select the second table (index 1)
            yield_curve_df = dfs[1]
        else:
            # Handle the case where there are fewer than two tables
            yield_curve_df = dfs[0]
        
        # You can process the dataframe further as needed
        return yield_curve_df
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

# Base URL without the last number
base_url = "https://www.federalreserve.gov/data/yield-curve-tables/feds200628_"


dataframes = []

for i in range(1, 14):
    url = f"{base_url}{i}.html"
    yield_curve_data = fetch_yield_curve_data(url)
    
    if yield_curve_data is not None:
        
        dataframes.append(yield_curve_data)

all_data = pd.concat(dataframes, ignore_index=True)
all_data.set_index('Date', inplace=True)

## Dataframe processing - change column names, sort
columns_required = ['SVENY01','SVENY02','SVENY03','SVENY04','SVENY05','SVENY06','SVENY07','SVENY08','SVENY09','SVENY10']

all_data = all_data[columns_required]
all_data.columns = [12, 24, 36, 48, 60, 72, 84, 96, 108, 120]
all_data.index = pd.to_datetime(all_data.index)
all_data= all_data.sort_index()

all_data

/var/folders/f1/z_r2bcjd6g59_5_526nb6g040000gn/T/ipykernel_60738/929845365.py:47: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  all_data.index = pd.to_datetime(all_data.index)


,12,24,36,48,60,72,84,96,108,120
Date,,,,,,,,,,
1999-03-04,5.025451,5.191532,5.270235,5.321379,5.370691,5.426934,5.490974,5.560452,5.632084,5.702713
1999-03-05,4.984128,5.126877,5.200250,5.252819,5.305292,5.364419,5.430436,5.500936,5.572803,5.643092
1999-03-08,4.968911,5.108597,5.176305,5.224118,5.273762,5.331977,5.398647,5.470957,5.545434,5.618858
1999-03-09,4.895288,5.017480,5.078848,5.126788,5.179954,5.243177,5.315212,5.392625,5.471639,5.548915
1999-03-10,4.888444,5.027980,5.100917,5.154699,5.209395,5.271397,5.340738,5.414899,5.490691,5.565122
...,...,...,...,...,...,...,...,...,...,...
2024-02-12,4.810100,4.411000,4.216900,4.128700,4.097100,4.096200,4.112300,4.137500,4.167400,4.199300
2024-02-13,4.943200,4.593600,4.405300,4.309800,4.268100,4.258100,4.266500,4.285300,4.309900,4.337200
2024-02-14,4.891200,4.526600,4.339100,4.248800,4.212900,4.207800,4.220200,4.242400,4.269700,4.299300


In [4]:
all_data = all_data.resample('M').last()
all_data

,12,24,36,48,60,72,84,96,108,120
Date,,,,,,,,,,
1999-03-31,4.842036,4.990334,5.072621,5.136951,5.202977,5.276463,5.356904,5.441409,5.526577,5.609324
1999-04-30,4.894796,5.058441,5.135048,5.188477,5.244374,5.310205,5.385473,5.466752,5.550039,5.631748
1999-05-31,5.120386,5.394610,5.528567,5.602364,5.655144,5.704252,5.755870,5.810815,5.867618,5.924093
1999-06-30,5.240925,5.479818,5.579605,5.645997,5.713296,5.788243,5.868316,5.948926,6.025949,6.096385
1999-07-31,5.307095,5.581700,5.723178,5.811520,5.882379,5.949648,6.017101,6.084243,6.149138,6.209688
...,...,...,...,...,...,...,...,...,...,...
2023-10-31,5.324300,5.020900,4.861800,4.791300,4.774700,4.790000,4.823100,4.865200,4.910600,4.955900
2023-11-30,5.084900,4.676100,4.446800,4.330500,4.284900,4.282800,4.306500,4.344500,4.389400,4.436500
2023-12-31,4.709400,4.213200,3.976800,3.868600,3.826300,3.818900,3.830400,3.852100,3.879100,3.908400


In [6]:

extrapolated_rates_df = pd.DataFrame(columns=np.arange(3, 121, 3), index=all_data.index)  # Set column names and index

months = np.array(all_data.columns)
for index, row in all_data.iterrows():
    # Extract rates
    rates = row[months].values
    # Fit a cubic spline to the data
    cs = CubicSpline(months, rates, bc_type='natural')

    # Use the spline to extrapolate quarterly rates
    quarterly_maturities = np.arange(3, 121, 3)  # From 3 to 120 months, every 3 months
    extrapolated_rates = cs(quarterly_maturities)
    # Append new row to the DataFrame
    extrapolated_rates_df.loc[index] = extrapolated_rates

extrapolated_rates_df

,3,6,9,12,15,18,21,24,27,30,...,93,96,99,102,105,108,111,114,117,120
Date,,,,,,,,,,,,,,,,,,,,,
1999-03-31,4.725431,4.761738,4.801118,4.842036,4.882953,4.922334,4.95864,4.990334,5.016386,5.037787,...,5.420072,5.441409,5.462801,5.484176,5.50546,5.526577,5.547475,5.568192,5.588789,5.609324
1999-04-30,4.764959,4.804855,4.848811,4.894796,4.940782,4.984737,5.024634,5.058441,5.084799,5.105027,...,5.446074,5.466752,5.487566,5.508439,5.52929,5.550039,5.570626,5.591076,5.611435,5.631748
1999-05-31,4.903678,4.970657,5.043944,5.120386,5.196829,5.270116,5.337095,5.39461,5.44047,5.476334,...,5.796817,5.810815,5.82494,5.839148,5.85339,5.867618,5.881792,5.895919,5.910014,5.924093
1999-06-30,5.050306,5.108394,5.173024,5.240925,5.308826,5.373456,5.431544,5.479818,5.516106,5.542624,...,5.928934,5.948926,5.968712,5.988205,6.007315,6.025949,6.044055,6.061735,6.079131,6.096385
1999-07-31,5.090584,5.157727,5.230903,5.307095,5.383287,5.456463,5.523606,5.5817,5.628666,5.666166,...,6.067563,6.084243,6.100803,6.117183,6.133317,6.149138,6.164607,6.179791,6.194787,6.209688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-31,5.562743,5.488449,5.407931,5.3243,5.240669,5.160151,5.085857,5.0209,4.967535,4.924585,...,4.854178,4.8652,4.876418,4.887765,4.89918,4.9106,4.921975,4.933307,4.944611,4.9559
2023-11-30,5.405003,5.304732,5.196745,5.0849,4.973055,4.865068,4.764797,4.6761,4.601781,4.540432,...,4.334114,4.3445,4.355302,4.366436,4.377828,4.3894,4.401084,4.412846,4.42466,4.4365
2023-12-31,5.101736,4.98057,4.847869,4.7094,4.570931,4.43823,4.317064,4.2132,4.130698,4.066784,...,3.846031,3.8521,3.858488,3.865148,3.872033,3.8791,3.886305,3.893613,3.90099,3.9084


In [7]:
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
import requests
import fred_rates_3_6
import one_year_rates

short_term_rates = fred_rates_3_6.pull_fred_data('2001-01-02','2024-02-15')
long_term_rates = 

def extrapolate_rates(rates):
    extrapolated_rates_df = pd.DataFrame(columns=np.arange(3, 121, 3), index=rates.index)  # Set column names and index

    months = np.array(rates.columns)
    for index, row in all_data.iterrows():
        # Extract rates for a date
        rates = row[months].values
        # Fit a cubic spline to the data
        cs = CubicSpline(months, rates, bc_type='natural')

        # Use the spline to extrapolate quarterly rates
        quarterly_maturities = np.arange(3, 121, 3)  # From 3 to 120 months, every 3 months
        extrapolated_rates = cs(quarterly_maturities)
        # Append new row to the DataFrame
        extrapolated_rates_df.loc[index] = extrapolated_rates

    return extrapolated_rates_df

In [8]:
df = extrapolate_rates(all_data)
df

,3,6,9,12,15,18,21,24,27,30,...,93,96,99,102,105,108,111,114,117,120
Date,,,,,,,,,,,,,,,,,,,,,
1999-03-31,4.725431,4.761738,4.801118,4.842036,4.882953,4.922334,4.95864,4.990334,5.016386,5.037787,...,5.420072,5.441409,5.462801,5.484176,5.50546,5.526577,5.547475,5.568192,5.588789,5.609324
1999-04-30,4.764959,4.804855,4.848811,4.894796,4.940782,4.984737,5.024634,5.058441,5.084799,5.105027,...,5.446074,5.466752,5.487566,5.508439,5.52929,5.550039,5.570626,5.591076,5.611435,5.631748
1999-05-31,4.903678,4.970657,5.043944,5.120386,5.196829,5.270116,5.337095,5.39461,5.44047,5.476334,...,5.796817,5.810815,5.82494,5.839148,5.85339,5.867618,5.881792,5.895919,5.910014,5.924093
1999-06-30,5.050306,5.108394,5.173024,5.240925,5.308826,5.373456,5.431544,5.479818,5.516106,5.542624,...,5.928934,5.948926,5.968712,5.988205,6.007315,6.025949,6.044055,6.061735,6.079131,6.096385
1999-07-31,5.090584,5.157727,5.230903,5.307095,5.383287,5.456463,5.523606,5.5817,5.628666,5.666166,...,6.067563,6.084243,6.100803,6.117183,6.133317,6.149138,6.164607,6.179791,6.194787,6.209688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-31,5.562743,5.488449,5.407931,5.3243,5.240669,5.160151,5.085857,5.0209,4.967535,4.924585,...,4.854178,4.8652,4.876418,4.887765,4.89918,4.9106,4.921975,4.933307,4.944611,4.9559
2023-11-30,5.405003,5.304732,5.196745,5.0849,4.973055,4.865068,4.764797,4.6761,4.601781,4.540432,...,4.334114,4.3445,4.355302,4.366436,4.377828,4.3894,4.401084,4.412846,4.42466,4.4365
2023-12-31,5.101736,4.98057,4.847869,4.7094,4.570931,4.43823,4.317064,4.2132,4.130698,4.066784,...,3.846031,3.8521,3.858488,3.865148,3.872033,3.8791,3.886305,3.893613,3.90099,3.9084
